In [15]:
import os
import shutil
import pathlib
import gc
import pandas as pd
import ttach as tta
from pathlib import Path
from PIL import Image
import numpy as np
import cv2 as cv
import random
import matplotlib.pyplot as plt
# from tqdm.auto import tqdm
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import torch
from torch.utils.data import DataLoader, random_split
from torch.utils.data import Dataset
from torch.optim.swa_utils import AveragedModel, SWALR

import torchvision
from torchvision import datasets
import cv2
from torch.cuda import amp

import torchvision.transforms as T
from torchvision.transforms import Compose, ToTensor, Resize
from torchvision.utils import make_grid
import optuna

import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
import timm

In [16]:
def seed_everything(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

In [17]:
seed_everything(42)

In [18]:
ensemple_type = "No" #"No" #"mine"

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [20]:
import torchvision.transforms as T


class ContrailDataset:
    def __init__(self, df, transform=None, normalize=False):
        self.df = df  
        self.images = df['image']
        self.labels = df['label']
        self.transform =transform
        self.normalize_image = T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        self.normalize=normalize
        
        
    def __len__(self):
        return len(self.images)
        
    def __getitem__(self, idx):
        image = np.load("../../input/" + self.images[idx]).astype(float)   
        label = np.load("../../input/" + self.labels[idx]).astype(float)
        

        # label_cls = 1 if label.sum() > 0 else 0
        if self.transform :
            data = self.transform(image=image, mask=label)
            image  = data['image']
            label  = data['mask']
            image = np.transpose(image, (2, 0, 1))
            label = np.transpose(label, (2, 0, 1))    
            
            
#         return torch.tensor(image), torch.tensor(label)
    
        if self.normalize:
            image = self.normalize_image(torch.tensor(image))
            return image, torch.tensor(label)
        else:
            return torch.tensor(image), torch.tensor(label)
    
    

In [21]:
###TImmunetplusplus model Nirjhar

n_blocks = 4

class TimmSegModel(nn.Module):
    def __init__(self, cfg, segtype='unet', pretrained=True):
        super(TimmSegModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3, stride=1, padding=1, bias=False)
        self.conv2 = nn.Conv2d(6, 12, 3, stride=1, padding=1, bias=False)
        self.conv3 = nn.Conv2d(12, 36, 3, stride=1, padding=1, bias=False)
        self.mybn1 = nn.BatchNorm2d(6)
        self.mybn2 = nn.BatchNorm2d(12)
        self.mybn3 = nn.BatchNorm2d(36)     
        self.encoder = timm.create_model(
            cfg['backbone'],
            in_chans=3,
            features_only=True,
            drop_rate=0.8,
            drop_path_rate=0.5,
            pretrained=False
        )
        self.encoder.conv_stem=nn.Conv2d(6, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

        self.encoder.blocks[5] = nn.Identity()
        self.encoder.blocks[6] = nn.Sequential(
            nn.Conv2d(self.encoder.blocks[4][2].conv_pwl.out_channels, 320, 1),
            nn.BatchNorm2d(320),
            nn.ReLU6(),
        )
        tr = torch.randn(1,6,64,64)
        g = self.encoder(tr)
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:n_blocks+1],
                decoder_channels=decoder_channels[:n_blocks],
                n_blocks=n_blocks,
            )

        self.segmentation_head = nn.Conv2d(
            decoder_channels[n_blocks-1],
            cfg["num_classes"], 
            kernel_size=(3, 3),
            stride=(1, 1), 
            padding=(1, 1)
        )

    def forward(self,x):
        x = F.relu6(self.mybn1(self.conv1(x)))
        global_features = [0] + self.encoder(x)[:n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features
    

def load_model1(cfg):
    model = TimmSegModel(cfg)
    model.load_state_dict(torch.load(cfg['model_pth']))
    return model


#### Model 2 Nirjhar

n_blocks =4
class TimmSegModel2(nn.Module):
    def __init__(self, cfg, segtype='unet', pretrained=True):
        super(TimmSegModel2, self).__init__()

        self.encoder = timm.create_model(
            cfg['backbone'],
            in_chans=3,
            features_only=True,
            drop_rate=0.5,
            pretrained=False
        )
        g = self.encoder(torch.rand(1, 3, 128, 128))
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:n_blocks+1],
                decoder_channels=decoder_channels[:n_blocks],
                n_blocks=n_blocks,
            )

        self.segmentation_head = nn.Sequential(
            nn.Conv2d(decoder_channels[n_blocks-1], 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.UpsamplingBilinear2d(scale_factor=1))

    def forward(self,x):
        global_features = [0] + self.encoder(x)[:n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features
    

def load_model2(cfg):
    model = TimmSegModel2(cfg)
    model.load_state_dict(torch.load(cfg['model_pth']))
    return model


##################

n_blocks = 4

class TimmSegModel3(nn.Module):
    def __init__(self, cfg, segtype='unet', pretrained=True):
        super(TimmSegModel3, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3, stride=1, padding=1, bias=False)
        self.conv2 = nn.Conv2d(6, 12, 3, stride=1, padding=1, bias=False)
        self.conv3 = nn.Conv2d(12, 36, 3, stride=1, padding=1, bias=False)
        self.mybn1 = nn.BatchNorm2d(6)
        self.mybn2 = nn.BatchNorm2d(12)
        self.mybn3 = nn.BatchNorm2d(36)     
        self.encoder = timm.create_model(
            cfg["backbone"],
            in_chans=3,
            features_only=True,
            drop_rate=0.8,
            drop_path_rate=0.5,
            pretrained=False
        )
        self.encoder.conv_stem=nn.Conv2d(6, 72, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

        self.encoder.blocks[5] = nn.Identity()
        self.encoder.blocks[6] = nn.Sequential(
            nn.Conv2d(self.encoder.blocks[4][2].conv_pwl.out_channels, 320, 1),
            nn.BatchNorm2d(320),
            nn.ReLU6(),
        )
        tr = torch.randn(1,6,64,64)
        g = self.encoder(tr)
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:n_blocks+1],
                decoder_channels=decoder_channels[:n_blocks],
                n_blocks=n_blocks,
            )

        self.segmentation_head = nn.Conv2d(decoder_channels[n_blocks-1], cfg['num_classes'], kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

    def forward(self,x):
        x = F.relu6(self.mybn1(self.conv1(x)))
        global_features = [0] + self.encoder(x)[:n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features
    
    
    
def load_model3(cfg):
    model = TimmSegModel3(cfg)
    model.load_state_dict(torch.load(cfg['model_pth']))
    return model



class TimmSegModel4(nn.Module):
    def __init__(self, cfg, segtype='unet', pretrained=True):
        super(TimmSegModel4, self).__init__()

        self.encoder = timm.create_model(
            cfg["backbone"],
            in_chans=3,
            features_only=True,
            drop_rate=0.5,
            pretrained=False
        )
        g = self.encoder(torch.rand(1, 3, 512, 512))
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:n_blocks+1],
                decoder_channels=decoder_channels[:n_blocks],
                n_blocks=n_blocks,
            )

        self.segmentation_head = nn.Sequential(
            nn.Conv2d(decoder_channels[n_blocks-1], 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.UpsamplingBilinear2d(scale_factor=1))

    def forward(self,x):
        global_features = [0] + self.encoder(x)[:n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features
    
    
def load_model4(cfg):
    model = TimmSegModel4(cfg)
    model.load_state_dict(torch.load(cfg['model_pth']))
    return model




n_blocks_1 = 5
class TimmSegModel5(nn.Module):
    def __init__(self, cfg, segtype='unet', pretrained=True):
        super(TimmSegModel5, self).__init__()

        self.encoder = timm.create_model(
            cfg["backbone"],
            in_chans=3,
            features_only=True,
            drop_rate=0.5,
            pretrained=False
        )
        g = self.encoder(torch.rand(1, 3, 512, 512))
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:n_blocks_1+1],
                decoder_channels=decoder_channels[:n_blocks_1],
                n_blocks=n_blocks_1,
            )

        self.segmentation_head = nn.Sequential(
            nn.Conv2d(decoder_channels[n_blocks_1-1], 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.UpsamplingBilinear2d(scale_factor=1))

    def forward(self,x):
        global_features = [0] + self.encoder(x)[:n_blocks_1]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features
    
    
def load_model5(cfg):
    model = TimmSegModel5(cfg)
    model.load_state_dict(torch.load(cfg['model_pth']))
    return model




## Rohit Model 
class Net_R(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        
        if cfg['model_name'] == 'Unet':
            self.model = smp.Unet(
                encoder_name=cfg["backbone"],     
                encoder_weights=None,   
                in_channels=3,  
                classes=cfg["num_classes"],
                activation=None
            )
            
        elif cfg['model_name'] == 'FPN':
            self.model = smp.FPN(
                encoder_name=cfg["backbone"],     
                encoder_weights=None,   
                in_channels=3,  
                classes=cfg["num_classes"],
                activation=None
            )
        
    
    def forward(self, inputs):
        mask = self.model(inputs)
        return mask
    
    
def load_modelr1(cfg):
    model = Net_R(cfg)
    model = torch.nn.DataParallel(model).cuda()
    model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])
    return model


def load_modelr1(cfg):
    model = Net_R(cfg)
    model = torch.nn.DataParallel(model).cuda()
    model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])
    return model


# def load_modelr3(cfg):
# #     weight = f"{cfg.architecture.pretrained_weights}"
#     model = Net_R(cfg)
#     model = torch.nn.DataParallel(model).cuda()
    
#     d =  torch.load(cfg['model_pth'], map_location="cpu")
#     if "model" in d:
#         model_weights = d["model"]
#     else:
#         model_weights = d
#     try:
#         model.load_state_dict(model_weights, strict=True)
#     except Exception as e:
#         print("removing unused pretrained layers")
#         for layer_name in re.findall("size mismatch for (.*?):", str(e)):
#             model_weights.pop(layer_name, None)
#         model.load_state_dict(model_weights, strict=False)

#     print(f"Weights loaded from: {cfg.architecture.pretrained_weights}")



#     if idx in [11, 12]:
#         model = TimmSegModelR1(cfg)
#         model = torch.nn.DataParallel(model).cuda()
#         model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])
#     else:
#         model = Net(cfg)    
#         model = torch.nn.DataParallel(model).cuda()
#         model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])

class TimmSegModelR1(nn.Module):
    def __init__(self, cfg, segtype='unet', pretrained=True):
        super(TimmSegModelR1, self).__init__()

        self.n_blocks = 4
        self.encoder = timm.create_model(
            cfg['backbone'],
            in_chans=3,
            features_only=True,
            drop_rate=0.5,
            pretrained=False
        )
        g = self.encoder(torch.rand(1, 3, 128, 128))
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:self.n_blocks+1],
                decoder_channels=decoder_channels[:self.n_blocks],
                n_blocks=self.n_blocks,
            )

        self.segmentation_head = nn.Sequential(
            nn.Conv2d(decoder_channels[self.n_blocks-1], 
                      cfg['num_classes'],
                      kernel_size=(3, 3),
                      stride=(1, 1),
                      padding=(1, 1)
            ),
            nn.UpsamplingBilinear2d(scale_factor=1)
        )

    def forward(self,x):
        global_features = [0] + self.encoder(x)[:self.n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features


    
def load_modelr2(cfg):
    model = TimmSegModelR1(cfg)
    model = torch.nn.DataParallel(model).cuda()
    model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])
    return model

### Ioannis

In [22]:
def dice_coef(y_true, y_pred, thr=0.5, epsilon=1e-6):
    y_true = y_true.to(torch.float32)
    y_pred = (y_pred>thr).to(torch.float32)
    inter = (y_true*y_pred).sum()
    den = y_true.sum() + y_pred.sum()
    dice = ((2*inter+epsilon)/(den+epsilon)).mean()
    
    return dice


def get_transform(img_size):
    transform = A.Compose([
        A.Resize(*img_size, interpolation=cv2.INTER_NEAREST),
    ], p=1.0)
    return transform

def get_transform2(img_size):
    transform = A.Compose([
        A.Resize(*img_size, interpolation=cv2.INTER_LINEAR),
    ], p=1.0)
    return transform




In [23]:
val_df = pd.read_csv("../../input/data_utils/val_df_filled.csv")
val_df_full = val_df.copy()

# val_df = val_df.loc[val_df['class'] == 1].reset_index(drop=True)
val_dups = np.load("../../input/data_utils/dups_val.npy")

val_dups = [int(val_id) for val_id in val_dups]

In [24]:
# val_df = val_df.loc[~val_df['id'].isin(val_dups)].reset_index(drop=True)
print(val_df.shape, val_df_full.shape)

(1856, 4) (1856, 4)


In [25]:
# val_df.to_csv(f"../../output/final_preds/val_df_pos.csv", index=False)

In [26]:
CFGS1 = [
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output3/exp_10_s1_full/Unet/efficientnet-b7-512/checkpoint_dice_ctrl_fold0.pth',
        'threshold': 0.24, #0.24,
        'call_sign': "roh_10_tta_flip",
        'model_func': load_modelr1,
        'tta': True,
        'normalize': False
    },
    
]

In [14]:
final_preds1 = []

for idx, cfg in enumerate(CFGS1):   

#     if idx <= 10:
#         continue
        
    print(cfg)
    val_transform = get_transform(cfg['img_size'])
    val_transform2 = get_transform2(cfg['img_size'])

    valid_dataset = ContrailDataset(val_df, transform=val_transform, normalize=False)  
    if cfg['normalize'] and ("ioa_" in cfg['call_sign']):
        valid_dataset = ContrailDataset(val_df, transform=val_transform2, normalize=True)  
#         valid_dataset = ContrailDataset(val_df, transform=val_transform, normalize=True)  


    
    valid_loader = DataLoader(
        valid_dataset, 
        batch_size = 32, #32, 
        shuffle = False, 
        num_workers = 2, 
        pin_memory = True, 
        drop_last = False
    )
    
    
    model = cfg['model_func'](cfg)        
      
    if cfg['tta']:
        if "roh_" in cfg['call_sign']:

            model = tta.SegmentationTTAWrapper(model, tta.aliases.flip_transform(), merge_mode='mean')
#             model = tta.SegmentationTTAWrapper(model, tta.aliases.d4_transform(), merge_mode='mean')
#             model3 = tta.SegmentationTTAWrapper(model, tta.aliases.hflip_transform(), merge_mode='mean')

        else:
            model = tta.SegmentationTTAWrapper(model, tta.aliases.hflip_transform(), merge_mode='mean')
#             model = tta.SegmentationTTAWrapper(model, tta.aliases.flip_transform(), merge_mode='mean')
#             model = tta.SegmentationTTAWrapper(model, tta.aliases.d4_transform(), merge_mode='mean')


    
#     model1.to(device)
#     model2.to(device)
#     model3.to(device)
#     model1.eval()
#     model2.eval()
#     model3.eval()
    
    
    model.to(device)
    model.eval()
    
    
    preds = []
    masks_ = []      
        
    for index, (images, masks) in enumerate(tqdm(valid_loader)):  
        images  = images.to(device, dtype=torch.float)
        masks  = masks.to(device, dtype=torch.float)
        if cfg['img_size'][0] != 256:
            masks = torch.nn.functional.interpolate(masks, size=256, mode='nearest') 

        masks_.append(torch.squeeze(masks, dim=1))
        with torch.inference_mode():
            images = torch.nn.functional.interpolate(images, size=cfg['img_size'][0], mode='nearest')
            pred = model(images).sigmoid() 
#             pred2 = model2(images).sigmoid() 
#             pred3 = model3(images).sigmoid() 
            
#             pred = (pred1 + pred2 + pred3) / 3
                
            pred = torch.nn.functional.interpolate(pred, size=256, mode='nearest')
            preds.append(torch.squeeze(pred, dim=1))
        
            
            
    model_masks = torch.cat(masks_, dim=0)
    model_preds = torch.cat(preds, dim=0)
        
    print(model_preds.shape)

    np.save(f"../../output/final_preds/{cfg['call_sign']}", model_preds.detach().cpu().numpy())
#     if idx == 0:
#         np.save(f"../../output/final_preds/val_masks", model_masks.detach().cpu().numpy())

    
    best_threshold = 0.0
    best_dice_score = 0.0
    for threshold in [i / 100 for i in range(101)] :
        score = dice_coef(model_masks, model_preds, thr=threshold).cpu().detach().numpy() 
        if score > best_dice_score:
            best_dice_score = score
            best_threshold = threshold
    
        
    print(best_dice_score, best_threshold)
    final_preds1.append(model_preds)
    
#     del model1, model2, model3
    del model
    torch.cuda.empty_cache()
    gc.collect()

{'model_name': 'Unet', 'backbone': 'efficientnet-b7', 'img_size': [512, 512], 'num_classes': 1, 'model_pth': '/home/rohits/pv1/Contrail_Detection/output3/exp_10_s1_full/Unet/efficientnet-b7-512/checkpoint_dice_ctrl_fold0.pth', 'threshold': 0.24, 'call_sign': 'roh_10_tta_flip', 'model_func': <function load_modelr1 at 0x7faa1ef835b0>, 'tta': True, 'normalize': False}


  0%|          | 0/58 [00:00<?, ?it/s]

torch.Size([1856, 256, 256])
0.67817783 0.19


In [17]:
### 0.67995                                                                                                           

In [84]:
def get_small_area(wt, wt_all, mask):
    best_threshold = 0.0
    best_dice_score = 0.0
    best_area = 0.0
    for i in range(500):
#         wt1 = wt
#         for idx, val in enumerate(wt1):
#             if val.sum() <= i:
#                 wt1[idx, :, :] = 0
                
        wt1 = wt
        for idx, val in enumerate(wt_all):
            if val.sum() <= i:
                wt1[idx, :, :] = 0
                
        wt1 = wt1.cuda()
        for threshold in [i / 100 for i in range(101)] :
            score = dice_coef(model_masks, wt1, thr=threshold).cpu().detach().numpy() 
            if score > best_dice_score:
                best_dice_score = score
                best_threshold = threshold
                best_area = i
        print(best_dice_score, best_threshold, best_area)
                
    print(best_dice_score, best_threshold, best_area)    
    
    
    

In [115]:

# call_signs1 = [  
# #     "roh_02_tta",
# #     "roh_11_tta",
# #     "nir_05_tta",
#     "roh_11_tta", # 12527945585979453
#     "nir_06_tta", # 4274708716604963
#     "nir_09_tta", # 60359403891758
#     "121_tta", # 598088267045536
#     "162_tta", # 8735733730972042
#     "166_tta", # 2509448081151351
# #     "125_tta", 
# #     "196_tta",
# #     "204",
#     "229_tta",  # 4990068077158164
#     "235_tta", # 9475873953049087
#     "ioa_exp55_tta", # 5673029916578154

# #     "242_tta"
    
# ] 

# call_signs1 = [
# #     "roh_10_tta_flip",
#     "roh_11_tta", # d4
#     "nir_05",
#     "nir_06_tta",
#     "nir_09",
    
#     "121_tta",
#     "162_tta",
#     "166_tta",
#     "125",
#     "196",
# #     "127",
    
    
#     "229_tta",
#     "235_tta",
#     "ioa_exp55_tta"
# ]

# call_signs1 = [
# #     "roh_10_tta_flip",
#     "roh_11_tta", # d4
#     "nir_05",
#     "nir_09",
    
#     "121_tta",
#     "162_tta",
#     "166_tta",
#     "125",
#     "196",
# #     "127",
    
# #     "229_tta",
# #     "235_tta",
#     "ioa_exp55_tta",
# #     "nir_06_tta",

# ]


call_signs1 = [
    "235_tta", "nir_09_tta", "166_tta", "127_tta", "nir_06_tta", "121_tta", "roh_05_tta", 
    "121", "162", "nir_01"
]


model_masks = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/val_masks.npy")).cuda() #.flatten().cuda() 

preds = []
for idx, sign in tqdm(enumerate(call_signs1), total=len(call_signs1)):
    wt = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/oofs/{sign}.npy")).cuda() 
    preds.append(wt)    
    score = dice_coef(model_masks, wt, thr=0.5).cpu().detach().numpy() 
    print(score)
#     best_threshold = 0.0
#     best_dice_score = 0.0
#     for threshold in [i / 100 for i in range(101)] :
#         score = dice_coef(model_masks, wt, thr=threshold).cpu().detach().numpy() 
#         if score > best_dice_score:
#             best_dice_score = score
#             best_threshold = threshold
#     print(best_dice_score, best_threshold)
    
    
    
final_preds = torch.stack(preds).mean(dim=0)
score = dice_coef(model_masks, final_preds, thr=0.5).cpu().detach().numpy() 
print("Mean Ensemble CV", score)


  0%|          | 0/10 [00:00<?, ?it/s]

0.6838827
0.67202777
0.67855626
0.68252563
0.6752145
0.68600667
0.68670255
0.6707117
0.6730113
0.6578587
Mean Ensemble CV 0.69696844


In [116]:
final_preds = preds
final_preds = torch.stack(final_preds).mean(dim=0)
score = dice_coef(model_masks, final_preds, thr=0.5).cpu().detach().numpy() 

print("0.5 TH Score: ", score)


best_threshold = 0.0
best_dice_score = 0.0
for threshold in [i / 100 for i in range(101)] :
    if threshold < 0.35 or threshold > 0.51:
        continue
    
    score = dice_coef(model_masks, final_preds, thr=threshold).cpu().detach().numpy() 
    if score > best_dice_score:
        best_dice_score = score
        best_threshold = threshold
        
print(best_dice_score, best_threshold)

0.5 TH Score:  0.69696844
0.6990131 0.41


In [114]:
call_signs2 = [
#     "roh_12_2_tta"
#     "roh_05_tta",
#     "roh_12_2_tta"
    
#     "roh_10_tta_flip",
#     "roh_10_tta",
#     "roh_11_tta", # d4
#     "nir_05_tta",
#     "121_tta",
#     "162_tta",
#     "125_tta",
#     "196_tta",
#     "229_tta",
#     "235_tta",
#     "ioa_exp55_tta",
    
#     "127",
#     "162",
#     "125",
#     "196"
    
]

preds2 = []
for idx, sign in tqdm(enumerate(call_signs2), total=len(call_signs2)):
    wt = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/oofs/{sign}.npy")).cuda() 
    wt = preds + [wt]
    
    final_preds = torch.stack(wt).mean(dim=0)
    score = dice_coef(model_masks, final_preds, thr=0.5).cpu().detach().numpy() 
    print(idx, score)

0it [00:00, ?it/s]

In [ ]:
# 0.69445986

final_preds = preds
final_preds = torch.stack(final_preds).mean(dim=0)
score = dice_coef(model_masks, final_preds, thr=0.5).cpu().detach().numpy() 

print("0.5 TH Score: ", score)

In [85]:

# # call_signs1 = [  
# # #     "roh_02_tta",
# # #     "roh_11_tta",
# # #     "nir_05_tta",
# #     "roh_11_tta", # 12527945585979453
# #     "nir_06_tta", # 4274708716604963
# #     "nir_09_tta", # 60359403891758
# #     "121_tta", # 598088267045536
# #     "162_tta", # 8735733730972042
# #     "166_tta", # 2509448081151351
# # #     "125_tta", 
# # #     "196_tta",
# # #     "204",
# #     "229_tta",  # 4990068077158164
# #     "235_tta", # 9475873953049087
# #     "ioa_exp55_tta", # 5673029916578154

# # #     "242_tta"
    
# # ] 

# # call_signs1 = [
# # #     "roh_10_tta_flip",
# #     "roh_11_tta", # d4
# #     "nir_05",
# #     "nir_06_tta",
# #     "nir_09",
    
# #     "121_tta",
# #     "162_tta",
# #     "166_tta",
# #     "125",
# #     "196",
# # #     "127",
    
    
# #     "229_tta",
# #     "235_tta",
# #     "ioa_exp55_tta"
# # ]

# call_signs1 = [
# #     "roh_10_tta_flip",
#     "roh_11_tta", # d4
#     "nir_05",
#     "nir_09",
    
#     "121_tta",
#     "162_tta",
#     "166_tta",
#     "125",
#     "196",
# #     "127",
    
# #     "229_tta",
# #     "235_tta",
#     "ioa_exp55_tta",
# #     "nir_06_tta",

# ]




# # 198, 195, 184, 174, 150, 138

# thresholds = [
#     0.35, 0.51, 0.44, 0.37, 0.46
# ]


# model_masks = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/val_masks.npy")).cuda() #.flatten().cuda() 

# preds = []

# sharp = 0.55
# areas = [31, 15, 22, 31, 8, 10, 42, 13]

# for idx, sign in tqdm(enumerate(call_signs1), total=len(call_signs1)):
# #     if idx == 0:
# #         wt = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/{sign}.npy")).cuda()
# #     else:

    
#     wt = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/oofs/{sign}.npy")).cuda() 
    
# #     if idx == 9:
# #         wt = wt.flatten()
    
# #     for i, val in enumerate(wt):
# #         if val.sum() <= areas[idx]:
# #             wt[i, :, :] = 0
    
    
# #     for i, val in enumerate(wt):
# #         if val.sum() <= areas[idx]:
# #             wt[i, :, :] = 0
    
    
# #     wt1 = (wt + t3_preds + t5_preds) / 3
    
# #     get_small_area(wt, wt1, model_masks)
    
    
# #     wt = wt.flatten().cuda()
    
    
# #     wt = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/{sign}.npy")).flatten().cuda()
# #     if idx in [0, 3]:
# #     wt = wt**sharp
# #     wt = torch.where(wt <= thresholds[idx], 0.0, wt)

# #     print(wt.shape)

#     preds.append(wt)    
#     score = dice_coef(model_masks, wt, thr=0.5).cpu().detach().numpy() 
# #     print(score)
#     best_threshold = 0.0
#     best_dice_score = 0.0
#     for threshold in [i / 100 for i in range(101)] :
#         score = dice_coef(model_masks, wt, thr=threshold).cpu().detach().numpy() 
#         if score > best_dice_score:
#             best_dice_score = score
#             best_threshold = threshold
#     print(best_dice_score, best_threshold)
    
    
    
    
# #     print(score)
    
    
# # # preds1.append(preds2)
    
# final_preds = preds
# final_preds = torch.stack(final_preds).mean(dim=0)
# score = dice_coef(model_masks, final_preds, thr=0.5).cpu().detach().numpy() 

# print("0.5 TH Score: ", score)


# best_threshold = 0.0
# best_dice_score = 0.0
# for threshold in [i / 100 for i in range(101)] :
#     if threshold < 0.35 or threshold > 0.51:
#         continue
    
#     score = dice_coef(model_masks, final_preds, thr=threshold).cpu().detach().numpy() 
#     if score > best_dice_score:
#         best_dice_score = score
#         best_threshold = threshold
        
# print(best_dice_score, best_threshold)

  0%|          | 0/9 [00:00<?, ?it/s]

0.6448363 0.77
0.67100537 0.91
0.6681824 0.94
0.6868992 0.44
0.6853779 0.37
0.678758 0.46
0.65278506 0.58
0.6745524 0.41
0.6665687 0.3


In [86]:
def boost_dice(params, TH=None):    
    
    for index, val in enumerate(params.keys()):
        if index == 0:            
            preds = params[val]*final_preds[0]
        else:
            preds += params[val]*final_preds[index]
    
    param_sum = 0
    for key, val in params.items():
        param_sum += val

    preds = preds/param_sum
    
    
    
#     for i, val in enumerate(preds):
#         if val.sum() <= 28:
#             preds[i, :, :] = 0
    
#     print(preds.shape, model_masks.shape)


#     wt1 = (preds)
    get_small_area(preds, preds, model_masks)
#     print(preds.shape)    
    
    
    if TH:
        best_threshold = TH
        preds1 = (preds).double()
        best_score = dice_coef(model_masks, preds1, thr=TH).cpu().detach().numpy() # get_score(labels, preds) 
    else:
        best_threshold = 0.0
        best_score = 0.0
        for threshold in [i / 100 for i in range(101)] :              
            if threshold < 0.33 or threshold > 0.45:
                continue
#         for threshold in [0.37, 0.38, 0.39, 0.4]:              

            preds1 = ((preds)>threshold).double()            
            score = dice_coef(model_masks, preds1, thr=threshold).cpu().detach().numpy() # get_score(labels, preds) 
            if score > best_score:
                best_score = score
                best_threshold = threshold
            
#     best_score = score
#     best_threshold = threshold
    return best_score, best_threshold

In [87]:
def objective(trial):
    TH = 0.4 #0.38 #0.38 #0.38 #0.39
    
        
    params = {}    
#     for i in range(len(final_preds)):
#         params[f"w{i+1}"] = trial.suggest_float(f'w{i+1}', 0, 1)

    
    
    
    params[f"w1"] = 0.38479116100428423
#     params[f"w1"] = trial.suggest_float(f'w1', 0, 1)
    params[f"w2"] = 0.21230927974890823
#     params[f"w2"] = trial.suggest_float(f'w2', 0, 1)
    params[f"w3"] = 0.5382785001584371
#     params[f"w3"] = trial.suggest_float(f'w3', 0, 1)

    params[f"w4"] = 0.42671162938028734
#     params[f"w4"] = trial.suggest_float(f'w4', 0, 1)

    params[f"w5"] = 0.652154140303207
#     params[f"w5"] = trial.suggest_float(f'w5', 0, 1)
    params[f"w6"] = 0.5657679697544893
#     params[f"w6"] = trial.suggest_float(f'w6', 0, 1)
    params[f"w7"] = 0.27094684341070685
#     params[f"w7"] = trial.suggest_float(f'w7', 0, 1)

    params[f"w8"] = 0.6632976852863441
#     params[f"w8"] = trial.suggest_float(f'w8', 0, 1)

    params[f"w9"] = 0.24664856457602563
#     params[f"w9"] = trial.suggest_float(f'w9', 0, 1)
#     params[f"w10"] = 0.02079886272699696
#     params[f"w10"] = trial.suggest_float(f'w10', 0, 1)
#     params[f"w11"] = 0.958547203069201
#     params[f"w11"] = trial.suggest_float(f'w11', 0, 1)
#     params[f"w12"] = 0.580634539865007
#     params[f"w12"] = trial.suggest_float(f'w12', 0, 1)


    
    
    score, best_threshold  = boost_dice(params, TH=TH)
    params['threshold'] = best_threshold
    if not TH:
        print(params)
        
    return score

In [88]:
study = optuna.create_study(direction='maximize') #0.7013314962387085
study.optimize(objective, n_trials=300)
best_params = study.best_params

[I 2023-08-07 23:26:46,063] A new study created in memory with name: no-name-bd53d5d8-945a-41b7-9aa1-4bb64dc8ca6e


0.7011266 0.4 0
0.7011266 0.4 0
0.7011266 0.4 0
0.7011266 0.4 0
0.7011266 0.4 0
0.7011266 0.4 0
0.7011266 0.4 0
0.7011266 0.4 0
0.7011266 0.4 0
0.7011266 0.4 0
0.7011281 0.4 10
0.7011281 0.4 10
0.7011281 0.4 10
0.7011281 0.4 10
0.7011281 0.4 10
0.7011281 0.4 10
0.7011281 0.4 10
0.7011281 0.4 10
0.7011281 0.4 10
0.7011281 0.4 10
0.7011281 0.4 10
0.7011281 0.4 10
0.7011448 0.4 22
0.7011448 0.4 22
0.70117724 0.4 24
0.70120746 0.4 25
0.70120746 0.4 25
0.70120746 0.4 25
0.70120746 0.4 25
0.70120746 0.4 25
0.701284 0.4 30
0.7013285 0.4 31
0.7013285 0.4 31
0.7013285 0.4 31
0.7013285 0.4 31
0.7013285 0.4 31
0.7013285 0.4 31
0.7013285 0.4 31
0.7013285 0.4 31
0.7013285 0.4 31
0.7013285 0.4 31
0.7013285 0.4 31
0.7013285 0.4 31
0.7013285 0.4 31
0.7013285 0.4 31
0.7013285 0.4 31
0.7013285 0.4 31
0.7013285 0.4 31
0.7013285 0.4 31
0.7013285 0.4 31
0.7013285 0.4 31
0.7013285 0.4 31
0.7013285 0.4 31
0.7013285 0.4 31
0.7013285 0.4 31
0.7013285 0.4 31
0.7013285 0.4 31


[W 2023-08-07 23:27:24,214] Trial 0 failed with parameters: {} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/rohits/anaconda3/envs/contrail/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_434984/2601949994.py", line 44, in objective
    score, best_threshold  = boost_dice(params, TH=TH)
  File "/tmp/ipykernel_434984/1828043191.py", line 25, in boost_dice
    get_small_area(preds, preds, model_masks)
  File "/tmp/ipykernel_434984/3300256640.py", line 18, in get_small_area
    score = dice_coef(model_masks, wt1, thr=threshold).cpu().detach().numpy()
KeyboardInterrupt
[W 2023-08-07 23:27:24,215] Trial 0 failed with value None.


KeyboardInterrupt: 

In [89]:
# 0.7015675

best_params # 0.0.0.7011265754699707

# # {'w1': 0.2223815786205381, 'w2': 0.1460101485299246, 'w3': 0.26070970307361535, 'w4': 0.4027457337153253, 'w5': 0.6526582421453944, 'w6': 0.2822502717916394}


# # params

# {'w1': 0.4274708716604963,
#  'w2': 0.5950793403696619,
#  'w3': 0.5981031818823078,
#  'w4': 0.9052807157209483,
#  'w5': 0.24903814629277216,
#  'w6': 0.5675462825594632,
#  'w7': 0.49869913358097817,
#  'w8': 0.9473968763799567,
#  'w9': 0.02603252721027552,
#  'w10': 0.11450732040441257}

params = {}
params[f"w1"] = 0.38479116100428423
#     params[f"w1"] = trial.suggest_float(f'w1', 0, 1)
params[f"w2"] = 0.21230927974890823
#     params[f"w2"] = trial.suggest_float(f'w2', 0, 1)
params[f"w3"] = 0.5382785001584371
#     params[f"w3"] = trial.suggest_float(f'w3', 0, 1)

params[f"w4"] = 0.42671162938028734
#     params[f"w4"] = trial.suggest_float(f'w4', 0, 1)

params[f"w5"] = 0.652154140303207
#     params[f"w5"] = trial.suggest_float(f'w5', 0, 1)
params[f"w6"] = 0.5657679697544893
#     params[f"w6"] = trial.suggest_float(f'w6', 0, 1)
params[f"w7"] = 0.27094684341070685
#     params[f"w7"] = trial.suggest_float(f'w7', 0, 1)

params[f"w8"] = 0.6632976852863441
#     params[f"w8"] = trial.suggest_float(f'w8', 0, 1)

params[f"w9"] = 0.24664856457602563
params

{'w1': 0.38479116100428423,
 'w2': 0.21230927974890823,
 'w3': 0.5382785001584371,
 'w4': 0.42671162938028734,
 'w5': 0.652154140303207,
 'w6': 0.5657679697544893,
 'w7': 0.27094684341070685,
 'w8': 0.6632976852863441,
 'w9': 0.24664856457602563}

In [102]:
params = {}
params[f"w1"] = 0.48892826876637613
#     params[f"w1"] = trial.suggest_float(f'w1', 0, 1)
params[f"w2"] = 0.39725111951465825
#     params[f"w2"] = trial.suggest_float(f'w2', 0, 1)
params[f"w3"] = 0.24460012359742106
#     params[f"w3"] = trial.suggest_float(f'w3', 0, 1)
params[f"w4"] = 0.7451789378499236
#     params[f"w4"] = trial.suggest_float(f'w4', 0, 1)
params[f"w5"] = 0.6134586590507985
#     params[f"w5"] = trial.suggest_float(f'w5', 0, 1)
params[f"w6"] = 0.688514109363038
#     params[f"w6"] = trial.suggest_float(f'w6', 0, 1)
params[f"w7"] = 0.38473516786706907
#     params[f"w7"] = trial.suggest_float(f'w7', 0, 1)
params[f"w8"] = 0.6643243522283139
#     params[f"w8"] = trial.suggest_float(f'w8', 0, 1)
params[f"w9"] = 0.8006651429446084
#     params[f"w9"] = trial.suggest_float(f'w9', 0, 1)
params[f"w10"] = 0.6461977860980533
#     params[f"w10"] = trial.suggest_float(f'w10', 0, 1)
params[f"w11"] = 0.958547203069201
#     params[f"w11"] = trial.suggest_float(f'w11', 0, 1)
params[f"w12"] = 0.580634539865007
#     params[f"w12"] = trial.suggest_float(f'w12', 0, 1)

In [103]:
params

{'w1': 0.48892826876637613,
 'w2': 0.39725111951465825,
 'w3': 0.24460012359742106,
 'w4': 0.7451789378499236,
 'w5': 0.6134586590507985,
 'w6': 0.688514109363038,
 'w7': 0.38473516786706907,
 'w8': 0.6643243522283139,
 'w9': 0.8006651429446084,
 'w10': 0.6461977860980533,
 'w11': 0.958547203069201,
 'w12': 0.580634539865007}

In [167]:
params = {}    
#     for i in range(len(final_preds)):
#         params[f"w{i+1}"] = trial.suggest_float(f'w{i+1}', 0, 1)

params[f"w1"] = 0.17983155241560006
#     params[f"w1"] = trial.suggest_float(f'w1', 0, 1)
params[f"w2"] = 0.3921968863625922
#     params[f"w2"] = trial.suggest_float(f'w2', 0, 1)
params[f"w3"] = 0.5467899983419907
#     params[f"w3"] = trial.suggest_float(f'w3', 0, 1)
params[f"w4"] = 0.770948318764219
#     params[f"w4"] = trial.suggest_float(f'w4', 0, 1)
params[f"w5"] = 0.9994494733078347
#     params[f"w5"] = trial.suggest_float(f'w5', 0, 1)
params[f"w6"] = 0.6290244452681286
#     params[f"w6"] = trial.suggest_float(f'w6', 0, 1)
params[f"w7"] = 0.48637848861057426
#     params[f"w7"] = trial.suggest_float(f'w7', 0, 1)
params[f"w8"] = 0.19103711312728738
#     params[f"w8"] = trial.suggest_float(f'w8', 0, 1)
params[f"w9"] = 0.6303399534920496
#     params[f"w9"] = trial.suggest_float(f'w9', 0, 1)
params

{'w1': 0.17983155241560006,
 'w2': 0.3921968863625922,
 'w3': 0.5467899983419907,
 'w4': 0.770948318764219,
 'w5': 0.9994494733078347,
 'w6': 0.6290244452681286,
 'w7': 0.48637848861057426,
 'w8': 0.19103711312728738,
 'w9': 0.6303399534920496}

In [72]:
# 0.37

# {'w1': 0.839014852140699, 'w2': 0.5011350487644706, 'w3': 0.5425134900557348, 'w4': 0.5452545452276221, 'w5': 0.6026139561761139, 'w6': 0.9831197346233449, 'w7': 0.8036481597462362, 'w8': 0.5819171235801198}
# 0.7003488540649414


# Trial 1 finished with value: 0.6991791129112244 and parameters: {'w1': 0.2047525215621725}. Best is trial 1 with value:
# 0.0.6996479034423828
# 0.36
# {
# 'w1': 0.36485377740463365,
# 'w2': 0.5716506827395362,
# 'w3': 0.84301760289886,
# 'w4': 0.5944703175254541,
# 'w5': 0.8096022495041699
# }. Best is trial 202 with value: 0.698759913444519.


params = {}    
#     for i in range(len(final_preds)):
#         params[f"w{i+1}"] = trial.suggest_float(f'w{i+1}', 0, 1)

params[f"w1"] = 0.24573090810759818
#     params[f"w1"] = trial.suggest_float(f'w1', 0, 1)

params[f"w2"] = 0.07968599547857583
#     params[f"w2"] = trial.suggest_float(f'w2', 0, 1)

params[f"w3"] = 0.3988612609534191
#     params[f"w3"] = trial.suggest_float(f'w3', 0, 1)

params[f"w4"] = 0.05807376299716156
#     params[f"w4"] = trial.suggest_float(f'w4', 0, 1)

params[f"w5"] = 0.8609716669901828
#     params[f"w5"] = trial.suggest_float(f'w5', 0, 1)
params[f"w6"] = 0.7386570274001879
#     params[f"w6"] = trial.suggest_float(f'w6', 0, 1)

params[f"w7"] = 0.9788696721148743
#     params[f"w7"] = trial.suggest_float(f'w7', 0, 1)
params[f"w8"] = 0.8270723521215965
#     params[f"w8"] = trial.suggest_float(f'w8', 0, 1)
params[f"w9"] = 0.5590153126204409
#     params[f"w9"] = trial.suggest_float(f'w9', 0, 1)
params[f"w10"] = 0.7265498790207406
#     params[f"w10"] = trial.suggest_float(f'w10', 0, 1)
params[f"w11"] = 0.41216752969852644
#     params[f"w11"] = trial.suggest_float(f'w11', 0, 1)
params[f"w12"] = 0.5864404449549478
#     params[f"w12"] = trial.suggest_float(f'w12', 0, 1)

params

{'w1': 0.24573090810759818,
 'w2': 0.07968599547857583,
 'w3': 0.3988612609534191,
 'w4': 0.05807376299716156,
 'w5': 0.8609716669901828,
 'w6': 0.7386570274001879,
 'w7': 0.9788696721148743,
 'w8': 0.8270723521215965,
 'w9': 0.5590153126204409,
 'w10': 0.7265498790207406,
 'w11': 0.41216752969852644,
 'w12': 0.5864404449549478}

In [135]:
# {
#     'w1': 0.7458107541498612,
#     'w2': 0.2900793805943048,
#     'w3': 0.40430935291387626,
#     'w4': 0.12087058264990641,
#     'w5': 0.5101955071950133,
#     'w6': 0.28429946034177606,
#     'w7': 0.364869611492901,
#     'w8': 0.10441135348615381,
#     'w9': 0.48784730233361917,
#     'w10': 0.9105341528815705
# }

 # . Best is trial 289 with value: 0.6934514045715332.


# roh_01_tta roh_02_tta, roh_07_tta, roh_06, roh_07, 
# roh_03_tta, roh_06_tta, roh_03, roh_05, nir_06_tta, nir_01, nir_02
# roh_04_tta, roh_01, nir_02_tta, 131
# roh_02, nir_03_tta, nir_04_tta, nir_05_tta, nir_06, 160
# roh_04, 166
# nir_04, 160_tta
# nir_03, 127

In [53]:
# params_pos = {
#  'w1': 0.655195343067219,
#  'w2': 0.2394705943310217,
#  'w3': 0.2927163844564814,
#  'w4': 0.4880033823349387,
#  'w5': 0.8387396842282611,
#  'w6': 0.9530480302035045,
#  'w7': 0.9573363671729265,
#  'w8': 0.6251416370174683,
#  'w9': 0.5371852737978187,
#  'w10': 0.4593912556686621,
#  'w11': 0.0,
#  'w12': 0.0
# }


# params_pos = {
#  'w1': 0.0,
#  'w2': 0.0,
#  'w3': 0.0,
#  'w4': 0.3074762360451622,
#  'w5': 0.0,
#  'w6': 0.12305169260913107,
#  'w7': 0.5363904743468609,
#  'w8': 0.0,
#  'w9': 0.8717967389387182,
#  'w10': 0.0,
#  'w11': 0.9798424243827817,
#  'w12': 0.7060773818241292
# }

params_pos = {
 'w1': 1.3387089493909258e-05,
 'w2': 0.7832907921668216,
 'w3': 0.1644769178230628,
 'w4': 0.00055845444967733,
 'w5': 0.2303089374666643,
 'w6': 0.05687648005878053,
 'w7': 0.5300038291077748,
 'w8': 0.24812342792556633,
 'w9': 0.06386465242721732,
 'w10': 0.2985146430301337,
 'w11': 0.9845085492621458,
 'w12': 0.5463258184950742
}



# params_all = {
#  'w1': 0.3154856556313033,
#  'w2': 0.2350959121391397,
#  'w3': 0.27103628342691727,
#  'w4': 0.6492473698434531,
#  'w5': 0.8397641633442563,
#  'w6': 0.8477025966292081,
#  'w7': 0.8096714084748394,
#  'w8': 0.4026009341206447,
#  'w9': 0.9969765714831549,
#  'w10': 0.39361450317658253,
#  'w11': 0.0,
#  'w12': 0.0
# }

params_all =  {
 'w1': 0.03644385570565463,
 'w2': 0.2254980335743082,
 'w3': 0.15865020296782992,
 'w4': 0.19357382597848447,
 'w5': 0.49930822369312966,
 'w6': 0.5592562127174933,
 'w7': 0.9150150266829656,
 'w8': 0.4294644367585147,
 'w9': 0.11137152838536817,
 'w10': 0.3808669569832525,
 'w11': 0.3506406085711101,
 'w12': 0.30010922179305505
}


model_masks = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/val_masks.npy")).cuda() #.flatten().cuda() 


In [54]:
len(final_preds)


def weighted_ensemble(params, final_preds):    
    for index, val in enumerate(params.keys()):
        if index == 0:
            preds = params[val]*final_preds[0]
        else:
            preds += params[val]*final_preds[index]
    
    
    param_sum = 0
    for key, val in params.items():
        param_sum += val

    preds = preds/param_sum
    return preds

In [61]:
# 0.70111567
for i in range(0, 300, 10): # 645
    preds_final = final_preds[0]

    for idx in range(final_preds[0].shape[0]):
        preds = [val[idx] for val in final_preds]
        a = weighted_ensemble(params_all, preds)
        if a.sum() > i:
            a = 0.9*a + 0.1*weighted_ensemble(params_pos, preds)
        preds_final[idx] = a

    preds_final = preds_final


    score = dice_coef(model_masks, preds_final, thr=0.5).cpu().detach().numpy() 
#     print(i, score)
    best_threshold = 0.0
    best_dice_score = 0.0
    for threshold in [i / 100 for i in range(101)] :
        score = dice_coef(model_masks, preds_final, thr=threshold).cpu().detach().numpy() 
        if score > best_dice_score:
            best_dice_score = score
            best_threshold = threshold
    print(i, best_dice_score, best_threshold)

0 0.69992656 0.37
10 0.6999279 0.37
20 0.6999333 0.37
30 0.6999667 0.37
40 0.69994384 0.37
50 0.69993037 0.37
60 0.69992834 0.37
70 0.69993585 0.38
80 0.69991124 0.38
90 0.6998961 0.38
100 0.69989496 0.38
110 0.69997627 0.38
120 0.6999885 0.38
130 0.70000225 0.38
140 0.7000122 0.38
150 0.7000219 0.38
160 0.7000002 0.38
170 0.70003974 0.38
180 0.7000359 0.38
190 0.70004225 0.38
200 0.7000551 0.38
210 0.70004904 0.38
220 0.70005196 0.38
230 0.70004743 0.38
240 0.7000549 0.38
250 0.70004654 0.38
260 0.7000391 0.38
270 0.700033 0.38
280 0.7000319 0.38
290 0.700026 0.38


In [ ]:
len(final_preds)

In [51]:
x = np.ones(final_preds[0].shape)

In [44]:
for idx, val in enumerate(call_signs1):
    print(idx+1, val)

1 nir_01_tta
2 nir_05
3 121
4 122
5 125
6 162
7 162_tta
8 166_tta
9 121_tta


In [93]:
params = {}    
#     for i in range(len(final_preds)):
#         params[f"w{i+1}"] = trial.suggest_float(f'w{i+1}', 0, 1)

#     params[f"w1"] = trial.suggest_float(f'w1', 0, 1)
params[f"w1"] = 0.2559174030552294
#     params[f"w2"] = trial.suggest_float(f'w2', 0, 1)
params[f"w2"] = 0.17464567024483976
#     params[f"w3"] = trial.suggest_float(f'w3', 0, 1)
params[f"w3"] = 0.2901265133088786
#     params[f"w4"] = trial.suggest_float(f'w4', 0, 1)
params[f"w4"] = 0.5116453708319416
#     params[f"w5"] = trial.suggest_float(f'w5', 0, 1)
params[f"w5"] = 0.7560514736860692
#     params[f"w6"] = trial.suggest_float(f'w6', 0, 1)
params[f"w6"] = 0.2927534853839116
params

{'w1': 0.2559174030552294,
 'w2': 0.17464567024483976,
 'w3': 0.2901265133088786,
 'w4': 0.5116453708319416,
 'w5': 0.7560514736860692,
 'w6': 0.2927534853839116}

In [ ]:
# Trial 145 finished with value: 0.6954816579818726 and parameters: 
# {'w1': 0.3800712934060794, 'w2': 0.19827057881403873, 'w3': 0.8639190435791337, 'w4': 0.8183765509720466, 'w5': 0.17899313642679757, 'w6': 0.024590770644496746, 'w7': 0.7862484916619368, 'w8': 0.992834740398046, 'w9': 0.3995791428881191}. 
# Best is trial 145 with value: 0.6954816579818726. # 0.4

In [ ]:
# [I 2023-07-16 19:49:38,527] Trial 258 finished with value: 0.6955569982528687 and parameters: 
# {'w1': 0.35630544690551724, 'w2': 0.3882333167896021, 'w3': 0.793029020160463, 'w4': 0.589733873322586, 'w5': 0.1238533606039253, 'w6': 0.3136224869358406, 'w7': 0.7445672749651168, 'w8': 0.9585857088778065, 'w9': 0.3523881046014743}. Best is trial 258 with value: 0.6955569982528687.
# 0.41

In [ ]:
# [I 2023-07-16 20:01:35,463] Trial 231 finished with value: 0.6953996419906616 and parameters: {'w1': 0.03615687143100031, 'w2': 0.45224010503148204, 'w3': 0.663721473024527, 'w4': 0.6371224751418889, 'w5': 0.03698106916893698, 'w6': 0.15052361112796397, 'w7': 0.5701940304803765, 'w8': 0.910815487921195, 'w9': 0.47052561326641523}. Best is trial 231 with value: 0.6953996419906616.
# 0.42

In [ ]:
# CFGS1 = [   
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'gluon_senet154',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'threshold': 0.5, #0.24,
#         'model_func': load_model_exp28,
#         'model_pth': '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1.ckpt',
#         'call_sign': "ioa_01",
#         'tta': True, #False, #True
#         'normalize': True
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'gluon_senet154',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'threshold': 0.5, #0.24,
#         'call_sign': "ioa_02",
#         'model_func': load_model_exp28_snapshot,
#         'tta': False, #True
#         'normalize': True,
#         'model_pth': [
#             ######## full train.csv 
#             '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1.ckpt',
#             '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1-v1.ckpt',
#             '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1-v2.ckpt',
#             ######### 4-skf split 
#         ]
#     }, 
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'gluon_senet154',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'threshold': 0.5, #0.24,
#         'call_sign': "roh_02",
#         'model_func': load_model_exp28_snapshot,        
#         'model_pth': '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1.ckpt',
#         'tta': False, #True
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'gluon_senet154',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'threshold': 0.5, #0.24,
#         'call_sign': "roh_03",
#         'model_func': load_model_exp28_snapshot,        
#         'model_pth': '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1-v1.ckpt',
#         'tta': False, #True
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'gluon_senet154',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'threshold': 0.5, #0.24,
#         'call_sign': "roh_04",
#         'model_func': load_model_exp28_snapshot,        
#         'model_pth': '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1-v2.ckpt',
#         'tta': False, #True
#     }, 
    
#         {
#         'model_name': 'Unet',
#         'backbone': 'gluon_senet154',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'threshold': 0.5, #0.24,
#         'call_sign': "ioa_02",
#         'model_func': load_model_exp28_snapshot,
#         'tta': False, #True
#         'normalize': True,
#         'model_pth': [
#             ######## full train.csv 
#             '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1.ckpt',
#             '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1-v1.ckpt',
#             '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1-v2.ckpt',
#             ######### 4-skf split 
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0.ckpt',
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0-v1.ckpt',
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0-v2.ckpt',
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0-v3.ckpt',
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0-v4.ckpt',
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0-v5.ckpt'
#         ]
#     }, 
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'gluon_senet154',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'threshold': 0.5, #0.24,
#         'call_sign': "ioa_02_tta",
#         'model_func': load_model_exp28_snapshot,
#         'tta': True, #True
#         'normalize': True,
#         'model_pth': [
#             ######## full train.csv 
#             '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1.ckpt',
#             '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1-v1.ckpt',
#             '/home/rohits/pv1/Contrail_Detection/output/ioannis/Unet-gluon_senet154_fold-1-v2.ckpt',
#             ######### 4-skf split 
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0.ckpt',
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0-v1.ckpt',
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0-v2.ckpt',
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0-v3.ckpt',
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0-v4.ckpt',
#             #     '/kaggle/input/contrails-exp28b-senet154-512/Unet-gluon_senet154_fold0-v5.ckpt'
#         ]
#     }, 
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/efficientnet-b7-256/checkpoint_dice_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_01_tta",
#         'model_func': load_modelr1,
#         'tta': True
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/efficientnet-b7-256/checkpoint_dice_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_01",
#         'model_func': load_modelr1,
#         'tta': False
#     }, 
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'cs3darknet_l',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/cs3darknet_l-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_02_tta",
#         'model_func': load_modelr2,
#         'tta': True
#     }, 
    
#         {
#         'model_name': 'Unet',
#         'backbone': 'cs3edgenet_x',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/cs3edgenet_x-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_03_tta",
#         'model_func': load_modelr2,
#         'tta': True
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'cs3sedarknet_l',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/cs3sedarknet_l-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_04_tta",
#         'model_func': load_modelr2,
#         'tta': True
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'ecaresnet101d',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/ecaresnet101d-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_05_tta",
#         'model_func': load_modelr2,
#         'tta': True
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'ecaresnet269d',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/ecaresnet269d-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_06_tta",
#         'model_func': load_modelr2,
#         'tta': True
#     }, 
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'fbnetv3_g',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/fbnetv3_g-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_07_tta",
#         'model_func': load_modelr2,
#         'tta': True
#     }, 
    
        
#     {
#         'model_name': 'Unet',
#         'backbone': 'tf_mixnet_l',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/tf_mixnet_l-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_08_tta",
#         'model_func': load_modelr2,
#         'tta': True
#     }, 
    
    

#     {
#         'model_name': 'Unet',
#         'backbone': 'maxvit_small_tf_512',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/maxvitfold0mix_maxvit_small_tf_512_best_epochstage2oof685cv657-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_01_tta",
#         'model_func': load_model4,
#         'tta': True
#     },
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'eca_nfnet_l1',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl1v2fold0_eca_nfnet_l1_best_epochstage2-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_02_tta",
#         'model_func': load_model2,
#         'tta': True
#     },
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b8',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/effb8modelv2fold0_tf_efficientnet_b8_best_epochstage2-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_03_tta", 
#         'model_func': load_model3,
#         'tta': True
#     },
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b7_ns',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/contrail_01/output/nirjhar/qishentrialv2fpn512_tf_efficientnet_b7_ns_best_epochcv650lb675-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_04_tta", 
#         'model_func': load_model1,
#         'tta': True
#     }, 
    
    
    
#         {
#         'model_name': 'Unet',
#         'backbone': 'eca_nfnet_l2',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl2v2fold0_eca_nfnet_l2_best_epochstage2-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_05_tta",
#         'model_func': load_model2,
#         'tta': True
#     },
#     {
#     'model_name': 'Unet++',
#         'backbone': 'densenetblur121d',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl2v2fold0_densenetblur121d_best_epochstage2-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_06_tta", 
#         'model_func': load_model2,
#         'tta': True
#     },
    
#     {
#     'model_name': 'Unet++',
#         'backbone': 'convnextv2_base',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/convnextfold0_convnextv2_base_best_epochstage2-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_07_tta", 
#         'model_func': load_model2,
#         'tta': True
#     },
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'maxvit_small_tf_512',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/maxvitfold0mix_maxvit_small_tf_512_best_epochstage2oof685cv657-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_01",
#         'model_func': load_model4,
#         'tta': False
#     },
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'eca_nfnet_l1',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl1v2fold0_eca_nfnet_l1_best_epochstage2-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_02",
#         'model_func': load_model2,
#         'tta': False
#     },
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b8',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/effb8modelv2fold0_tf_efficientnet_b8_best_epochstage2-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_03", 
#         'model_func': load_model3,
#         'tta': False
#     },
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b7_ns',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/contrail_01/output/nirjhar/qishentrialv2fpn512_tf_efficientnet_b7_ns_best_epochcv650lb675-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_04", 
#         'model_func': load_model1,
#         'tta': False
#     }, 
    
#         {
#         'model_name': 'Unet',
#         'backbone': 'eca_nfnet_l2',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl2v2fold0_eca_nfnet_l2_best_epochstage2-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_05",
#         'model_func': load_model2,
#         'tta': False
#     },
#     {
#     'model_name': 'Unet++',
#         'backbone': 'densenetblur121d',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl2v2fold0_densenetblur121d_best_epochstage2-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_06", 
#         'model_func': load_model2,
#         'tta': False
#     },
    
#     {
#     'model_name': 'Unet++',
#         'backbone': 'convnextv2_base',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/convnextfold0_convnextv2_base_best_epochstage2-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_07", 
#         'model_func': load_model2,
#         'tta': False
#     },
    
    
    
    
    
# ]

In [29]:
cfg = [       
    "roh_02_tta", 
] 


# model_masks = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/val_masks.npy")).flatten().cuda() 
model_masks = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/val_masks.npy")).flatten().cuda() 


preds = []

for idx, sign in tqdm(enumerate(cfg), total=len(cfg)):
    wt = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/oofs/{sign}.npy"))
    
    
#     wt = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/oofs/{sign}.npy")).flatten().cuda()  
    
    

  0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
wt.shape

torch.Size([1856, 256, 256])

In [ ]:
for idx, val in enumerate(wt):
#     print(val.sum() <= 10)
    if val.sum() <= 10:
        wt[idx, :, :] = 0
    
# print("mhsdbc")

Exception ignored in: <function tqdm.__del__ at 0x7f0e633e88b0>
Traceback (most recent call last):
  File "/home/rohits/anaconda3/envs/contrail/lib/python3.10/site-packages/tqdm/std.py", line 1144, in __del__
    def __del__(self):
KeyboardInterrupt: 


In [26]:
# np.sum(val)

In [ ]:
wt = wt.flatten().cuda()
score = dice_coef(model_masks, wt, thr=0.5).cpu().detach().numpy() 
print("0.5 TH Score: ", score)

best_threshold = 0.0
best_dice_score = 0.0
for threshold in [i / 100 for i in range(101)] :
    score = dice_coef(model_masks, wt, thr=threshold).cpu().detach().numpy() 
    if score > best_dice_score:
        best_dice_score = score
        best_threshold = threshold
print(best_dice_score, best_threshold)

In [ ]:
# 0.67922837

In [42]:
def boost_dice(params, TH=None):    
    
    for index, val in enumerate(params.keys()):
        if index == 0:            
            preds = params[val]*final_preds[0]
        else:
            preds += params[val]*final_preds[index]
    
    param_sum = 0
    for key, val in params.items():
        param_sum += val

    preds = preds/param_sum
    
    
    
#     for i, val in enumerate(preds):
#         if val.sum() <= 26:
#             preds[i, :, :] = 0
    
#     print(preds.shape, model_masks.shape)
#     get_small_area(preds, model_masks)
#     print(preds.shape)    
    
    
    if TH:
        best_threshold = TH
        preds1 = (preds).double()
        best_score = dice_coef(model_masks, preds1, thr=TH).cpu().detach().numpy() # get_score(labels, preds) 
    else:
        best_threshold = 0.0
        best_score = 0.0
        for threshold in [i / 100 for i in range(101)] :              
            if threshold < 0.3 or threshold > 0.65:
                continue

            preds1 = ((preds)>threshold).double()            
            score = dice_coef(model_masks, preds1, thr=threshold).cpu().detach().numpy() # get_score(labels, preds) 
            if score > best_score:
                best_score = score
                best_threshold = threshold
            
#     best_score = score
#     best_threshold = threshold
    return best_score, best_threshold

In [176]:
call_signs1 = [    
    "roh_10_tta",
#     "roh_02_tta", 
    "nir_05",
    "nir_06_tta",
    "nir_09",
    "121_tta",
    "162_tta",
    "166_tta",
    "125", 
    "196",
    "127",
    
    "204",
    "roh_11_tta_full",
    "roh_12_tta"
    
    # above this 
    
#     "195",
#     "198",
#     "184", # 1
#     "174",
#     "150",
#     "138"    
    
        
#     "195_tta",
#     "198_tta",
#     "184_tta",
#     "174_tta",
#     "150_tta",
#     "138_tta"
    
    
] 

# 

# call_signs1 = [        
#     "roh_10_tta",
# #     "roh_02_tta", 
#     "roh_11_tta_full",

    
#     "nir_05",
#     "nir_06_tta",
#     "nir_09",
#     "121_tta",
#     "162_tta",
#     "166_tta",
#     "125", 
#     "196",
#     "127",
    
#     "204",
    
# ]

call_signs1 = [    
#     "roh_10_tta",
#     "roh_02_tta", 
    "roh_12_tta",
    "roh_02_tta",
    "roh_11_tta",
    "nir_05_tta",
    "nir_06_tta",
    "nir_09_tta",
    "121_tta",
    "162_tta",
    "166_tta",
    "125_tta", 
    "196_tta",
#     "127_tta",
    "204", # tta
    
    "Unetpp-convnext_base_preds_tta_06638",
    "230_tta"
    
#     "Unet-tf_efficientnet_l2_ns_tta_06570"
    # above this 
    
#     "nir_09"
    
#     "195_tta",
#     "198_tta",
#     "184_tta",
#     "174_tta",
#     "150_tta",
#     "138_tta"
] 



val_df = pd.read_csv("../../input/data_utils/val_df_filled.csv")

ix = val_df.loc[val_df['class'] == 1].index.values

model_masks = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/val_masks.npy")).cuda() #.flatten().cuda() 
model_masks = model_masks[ix]
preds = []

sharp = 0.55
areas = [31, 15, 22, 31, 8, 10, 42, 13]

for idx, sign in tqdm(enumerate(call_signs1), total=len(call_signs1)):
#     if idx == 0:
#         wt = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/{sign}.npy")).cuda()
#     else:
    wt = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/oofs/{sign}.npy")).cuda() 

    wt = wt[ix]
    preds.append(wt)    
    score = dice_coef(model_masks, wt, thr=0.5).cpu().detach().numpy() 
    print(score)
    best_threshold = 0.0
    best_dice_score = 0.0
    for threshold in [i / 100 for i in range(101)] :
        score = dice_coef(model_masks, wt, thr=threshold).cpu().detach().numpy() 
        if score > best_dice_score:
            best_dice_score = score
            best_threshold = threshold
    print(best_dice_score, best_threshold)
    
final_preds = preds


  0%|          | 0/14 [00:00<?, ?it/s]

0.6919633
0.6930453 0.23
0.68611187
0.6876079 0.17
0.6951875
0.6969542 0.27
0.68547714
0.6856808 0.26
0.6857713
0.6858334 0.47
0.6802031
0.6809007 0.04
0.69100106
0.69290924 0.33
0.68723655
0.6917759 0.34
0.6870522
0.68754715 0.45
0.6658337
0.6699166 0.23
0.6490005
0.6866923 0.39
0.69341856
0.6979238 0.34
0.67109334
0.6762293 0.25
0.67154294
0.67987764 0.22


In [177]:
def objective(trial):
    TH = None #0.38 # None #0.39 # None #0.39
    
    params = {}    
    for i in range(len(final_preds)):
        params[f"w{i+1}"] = trial.suggest_float(f'w{i+1}', 0, 1)

    score, best_threshold  = boost_dice(params, TH=TH)
    params['threshold'] = best_threshold
    if not TH:
        print(params)
        
    return score

In [178]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=500)
best_params = study.best_params

[I 2023-08-06 00:50:17,733] A new study created in memory with name: no-name-6cc276e5-735b-443c-a91d-e6e96d7f0b78
[I 2023-08-06 00:50:17,812] Trial 0 finished with value: 0.7041952013969421 and parameters: {'w1': 0.9516440463846043, 'w2': 0.18536023186023476, 'w3': 0.2659322405435598, 'w4': 0.019524292847678648, 'w5': 0.3909675398134319, 'w6': 0.8755558032429971, 'w7': 0.2656580207775051, 'w8': 0.8437076999949501, 'w9': 0.20910231204265783, 'w10': 0.8196981673292376, 'w11': 0.8633740779703943, 'w12': 0.36222679040795336, 'w13': 0.8964159797671869, 'w14': 0.06434233306361858}. Best is trial 0 with value: 0.7041952013969421.
[I 2023-08-06 00:50:17,888] Trial 1 finished with value: 0.7067200541496277 and parameters: {'w1': 0.1741329627814514, 'w2': 0.3637229687358404, 'w3': 0.5163241380739675, 'w4': 0.25159357839765917, 'w5': 0.223388175226386, 'w6': 0.1792371883365298, 'w7': 0.7123641757129227, 'w8': 0.861405854635238, 'w9': 0.22036829269649172, 'w10': 0.0520023719783812, 'w11': 0.241888

{'w1': 0.9516440463846043, 'w2': 0.18536023186023476, 'w3': 0.2659322405435598, 'w4': 0.019524292847678648, 'w5': 0.3909675398134319, 'w6': 0.8755558032429971, 'w7': 0.2656580207775051, 'w8': 0.8437076999949501, 'w9': 0.20910231204265783, 'w10': 0.8196981673292376, 'w11': 0.8633740779703943, 'w12': 0.36222679040795336, 'w13': 0.8964159797671869, 'w14': 0.06434233306361858, 'threshold': 0.33}
{'w1': 0.1741329627814514, 'w2': 0.3637229687358404, 'w3': 0.5163241380739675, 'w4': 0.25159357839765917, 'w5': 0.223388175226386, 'w6': 0.1792371883365298, 'w7': 0.7123641757129227, 'w8': 0.861405854635238, 'w9': 0.22036829269649172, 'w10': 0.0520023719783812, 'w11': 0.2418886440736363, 'w12': 0.6563978389327186, 'w13': 0.31539302070193875, 'w14': 0.10562828566345861, 'threshold': 0.34}
{'w1': 0.7640414502974435, 'w2': 0.9355669885918019, 'w3': 0.7960278775309649, 'w4': 0.5914886111784843, 'w5': 0.878135190260129, 'w6': 0.6452693538117582, 'w7': 0.6057069363449151, 'w8': 0.42865869799597756, 'w9':

[I 2023-08-06 00:50:18,040] Trial 3 finished with value: 0.7048695087432861 and parameters: {'w1': 0.42560695185276753, 'w2': 0.6303487917280433, 'w3': 0.9149503531618902, 'w4': 0.6398841768357301, 'w5': 0.8826218110663334, 'w6': 0.6351741021228395, 'w7': 0.31278460931348695, 'w8': 0.51524658790212, 'w9': 0.8584181627265611, 'w10': 0.4051499424854753, 'w11': 0.6921540131641452, 'w12': 0.6047915903013135, 'w13': 0.24493770804634873, 'w14': 0.3718339560275391}. Best is trial 1 with value: 0.7067200541496277.
[I 2023-08-06 00:50:18,108] Trial 4 finished with value: 0.7043448090553284 and parameters: {'w1': 0.27042788755397085, 'w2': 0.15264930691119682, 'w3': 0.9049677062052287, 'w4': 0.5852928217627424, 'w5': 0.16280702993128027, 'w6': 0.7731232297240248, 'w7': 0.00724631397813591, 'w8': 0.6772622825884901, 'w9': 0.3966647169723333, 'w10': 0.001721973944338595, 'w11': 0.5224824258522838, 'w12': 0.06238627325413015, 'w13': 0.5848480699564877, 'w14': 0.6340903868655331}. Best is trial 1 wi

{'w1': 0.42560695185276753, 'w2': 0.6303487917280433, 'w3': 0.9149503531618902, 'w4': 0.6398841768357301, 'w5': 0.8826218110663334, 'w6': 0.6351741021228395, 'w7': 0.31278460931348695, 'w8': 0.51524658790212, 'w9': 0.8584181627265611, 'w10': 0.4051499424854753, 'w11': 0.6921540131641452, 'w12': 0.6047915903013135, 'w13': 0.24493770804634873, 'w14': 0.3718339560275391, 'threshold': 0.33}
{'w1': 0.27042788755397085, 'w2': 0.15264930691119682, 'w3': 0.9049677062052287, 'w4': 0.5852928217627424, 'w5': 0.16280702993128027, 'w6': 0.7731232297240248, 'w7': 0.00724631397813591, 'w8': 0.6772622825884901, 'w9': 0.3966647169723333, 'w10': 0.001721973944338595, 'w11': 0.5224824258522838, 'w12': 0.06238627325413015, 'w13': 0.5848480699564877, 'w14': 0.6340903868655331, 'threshold': 0.33}
{'w1': 0.9110713186870055, 'w2': 0.5733035364367786, 'w3': 0.7486294181216968, 'w4': 0.05290369506212278, 'w5': 0.47829872764921466, 'w6': 0.246948196008421, 'w7': 0.6827470461322497, 'w8': 0.7144172807803135, 'w9'

[I 2023-08-06 00:50:18,245] Trial 6 finished with value: 0.7045413255691528 and parameters: {'w1': 0.13818434034095894, 'w2': 0.5938108511442584, 'w3': 0.676536563800303, 'w4': 0.27024207167542713, 'w5': 0.1990450094881494, 'w6': 0.20621488280150635, 'w7': 0.32134266667049205, 'w8': 0.7785048360027191, 'w9': 0.08819379587515219, 'w10': 0.7320394818518283, 'w11': 0.11667728159215807, 'w12': 0.407815727664, 'w13': 0.12219011079426023, 'w14': 0.01206877715985144}. Best is trial 1 with value: 0.7067200541496277.
[I 2023-08-06 00:50:18,313] Trial 7 finished with value: 0.7044281363487244 and parameters: {'w1': 0.19044344431951787, 'w2': 0.13954847204018417, 'w3': 0.3480526139669389, 'w4': 0.7682962674000697, 'w5': 0.5456920199193035, 'w6': 0.012845082580896783, 'w7': 0.21492220267716522, 'w8': 0.582828686124472, 'w9': 0.67045704128292, 'w10': 0.89645628046281, 'w11': 0.03199632970931077, 'w12': 0.908827535376485, 'w13': 0.11512614458826576, 'w14': 0.9970404563284099}. Best is trial 1 with v

...

{'w1': 0.09479650376962895, 'w2': 0.1245215859016171, 'w3': 0.8545171820081917, 'w4': 0.40529198894873686, 'w5': 0.10341477045675022, 'w6': 0.22040323273946566, 'w7': 0.08755554459698962, 'w8': 0.49634126908096, 'w9': 0.16408243602284198, 'w10': 0.19025910010054997, 'w11': 0.9014874077224672, 'w12': 0.8918737521925076, 'w13': 0.47438287326524486, 'w14': 0.024496335157584384, 'threshold': 0.34}
{'w1': 0.02672462354329528, 'w2': 0.09036491620781857, 'w3': 0.7955563543192419, 'w4': 0.377974704995999, 'w5': 0.07172387706070478, 'w6': 0.3078278018823431, 'w7': 0.06423846965791813, 'w8': 0.5300061479430026, 'w9': 0.14520757135397136, 'w10': 0.2317533157357407, 'w11': 0.9727852774026131, 'w12': 0.8122635029506924, 'w13': 0.5089545891373952, 'w14': 0.055432172077678296, 'threshold': 0.34}


[I 2023-08-06 00:50:30,269] Trial 103 finished with value: 0.7087171077728271 and parameters: {'w1': 0.054767735973705746, 'w2': 0.0657331116800597, 'w3': 0.8265804867241022, 'w4': 0.3967749720437086, 'w5': 0.13609750098675466, 'w6': 0.26481289484930026, 'w7': 0.1285154794385803, 'w8': 0.5026644299416397, 'w9': 0.16649287100320548, 'w10': 0.2134757596885707, 'w11': 0.9225458733655213, 'w12': 0.8941322356591499, 'w13': 0.45703149489438505, 'w14': 0.1097639696631623}. Best is trial 90 with value: 0.7088056802749634.
[I 2023-08-06 00:50:30,398] Trial 104 finished with value: 0.7085239291191101 and parameters: {'w1': 0.047211637867458245, 'w2': 0.03076949240470428, 'w3': 0.8241879816341418, 'w4': 0.4733717229601343, 'w5': 0.13174378966058276, 'w6': 0.26661788980763157, 'w7': 0.12474548567844124, 'w8': 0.5030447077604503, 'w9': 0.24851255904264147, 'w10': 0.2967732525944409, 'w11': 0.9231492080874238, 'w12': 0.8609602967844029, 'w13': 0.46551783120141693, 'w14': 0.13421022700363078}. Best i

{'w1': 0.054767735973705746, 'w2': 0.0657331116800597, 'w3': 0.8265804867241022, 'w4': 0.3967749720437086, 'w5': 0.13609750098675466, 'w6': 0.26481289484930026, 'w7': 0.1285154794385803, 'w8': 0.5026644299416397, 'w9': 0.16649287100320548, 'w10': 0.2134757596885707, 'w11': 0.9225458733655213, 'w12': 0.8941322356591499, 'w13': 0.45703149489438505, 'w14': 0.1097639696631623, 'threshold': 0.33}
{'w1': 0.047211637867458245, 'w2': 0.03076949240470428, 'w3': 0.8241879816341418, 'w4': 0.4733717229601343, 'w5': 0.13174378966058276, 'w6': 0.26661788980763157, 'w7': 0.12474548567844124, 'w8': 0.5030447077604503, 'w9': 0.24851255904264147, 'w10': 0.2967732525944409, 'w11': 0.9231492080874238, 'w12': 0.8609602967844029, 'w13': 0.46551783120141693, 'w14': 0.13421022700363078, 'threshold': 0.33}


[W 2023-08-06 00:50:30,513] Trial 105 failed with parameters: {'w1': 0.057939296261058607, 'w2': 0.03899101317102122, 'w3': 0.7710923187971663, 'w4': 0.3408263867786405, 'w5': 0.0797882141702656, 'w6': 0.2588073974545686, 'w7': 0.08170499168291678, 'w8': 0.592748451450337, 'w9': 0.17560452538845325, 'w10': 0.25956200865738227, 'w11': 0.9397620930468926, 'w12': 0.8915036107518929, 'w13': 0.46049588392217744, 'w14': 0.11004003485238993} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/rohits/anaconda3/envs/contrail/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_3368091/798499834.py", line 8, in objective
    score, best_threshold  = boost_dice(params, TH=TH)
  File "/tmp/ipykernel_3368091/3617302815.py", line 40, in boost_dice
    preds1 = ((preds)>threshold).double()
KeyboardInterrupt
[W 2023-08-06 00:50:30,514] Trial 105 failed with value None.

KeyboardInterrupt: 

In [47]:
best_params # 0.36 | 0.710804283618927

{'w1': 1.3387089493909258e-05,
 'w2': 0.7832907921668216,
 'w3': 0.1644769178230628,
 'w4': 0.00055845444967733,
 'w5': 0.2303089374666643,
 'w6': 0.05687648005878053,
 'w7': 0.5300038291077748,
 'w8': 0.24812342792556633,
 'w9': 0.06386465242721732,
 'w10': 0.2985146430301337,
 'w11': 0.9845085492621458,
 'w12': 0.5463258184950742}

In [19]:
call_signs1 = [
    "/home/rohits/pv1/Contrail_Detection/output/pseudo_preds/roh_10_tta_3.pt",
    "/home/rohits/pv1/Contrail_Detection/output/pseudo_preds/roh_10_tta_4.npy",
    "/home/rohits/pv1/Contrail_Detection/output/pseudo_preds/roh_10_tta_5.pt"
]

model_masks = torch.from_numpy(np.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/val_masks.npy")).cuda() #.flatten().cuda() 


preds = []
for idx, sign in tqdm(enumerate(call_signs1), total=len(call_signs1)):
    if idx != 1:
        wt = torch.load(sign).cuda()
    else:
        wt = torch.from_numpy(np.load(sign)).cuda()
        
    preds.append(wt)

  0%|          | 0/3 [00:00<?, ?it/s]

In [20]:
model_masks.shape, preds[0].shape

(torch.Size([1856, 256, 256]), torch.Size([1856, 256, 256]))

In [21]:
for pred in preds:
    score = dice_coef(model_masks, pred, thr=0.5).cpu().detach().numpy() 
    print(score)
    best_threshold = 0.0
    best_dice_score = 0.0
    for threshold in [i / 100 for i in range(101)] :
        score = dice_coef(model_masks, pred, thr=threshold).cpu().detach().numpy() 
        if score > best_dice_score:
            best_dice_score = score
            best_threshold = threshold
    print(best_dice_score, best_threshold)

0.13172013
0.14183973 0.01
0.675066
0.67817783 0.19
0.12852824
0.13934006 0.01


In [63]:
x1 = (preds[0] > 0.19).double()
x2 = (preds[1] > 0.19).double()
x3 = (preds[2] > 0.19).double()

In [64]:
sum1 = []
for a in x1:
    sum1.append(a.sum().cpu().numpy().item())
    
    
sum2 = []
for a in x2:
    sum2.append(a.sum().cpu().numpy().item())
    
    
    
sum3 = []
for a in x3:
    sum3.append(a.sum().cpu().numpy().item())
    
    
sum4 = []
for a in model_masks:
    sum4.append(a.sum().cpu().numpy().item())

In [65]:
fps = []
for idx, (a, b, c, d) in enumerate(zip(sum1, sum2, sum3, sum4)):
    print(a, b, c, d)
    if idx > 1:
        break
    
    if a == b == c == 0 and d != 0:
        fps.append(d)
    

0.0 0.0 0.0 0.0
260.0 312.0 274.0 367.0
0.0 0.0 0.0 0.0


In [66]:
len(fps)

0

In [67]:
x1 = x1 #.flatten()
x2 = x2 #.flatten()
x3 = x3 #.flatten()

In [68]:
x = (x1 + x2 + x3)
    
    
x = (x >= 1) == 1

In [70]:
score = dice_coef(model_masks, x, thr=0.5).cpu().detach().numpy() 
score

array(0.39960405, dtype=float32)

In [62]:
x1[2], x2[2], x3[2]

(tensor(0., device='cuda:0', dtype=torch.float64),
 tensor(0., device='cuda:0', dtype=torch.float64),
 tensor(0., device='cuda:0', dtype=torch.float64))

In [89]:
# model_masks

import numpy as np

# def merge_predictions(mask_t_minus_1, mask_t, mask_t_plus_1, time_shift):
#     # Ensure all three masks have the same dimensions (256x256)
#     assert mask_t_minus_1.shape == mask_t.shape == mask_t_plus_1.shape == (256, 256), "All masks must have size 256x256"

#     # Create an empty merged mask with zeros
#     merged_mask = np.zeros((256, 256))

#     # Time shift should be within the bounds of the mask dimensions
#     time_shift = max(-1, min(1, time_shift))

#     # Calculate the shift directions for t-1, t, and t+1 masks
#     shift_t_minus_1 = (0, 0) if time_shift == 1 else (time_shift, time_shift)
#     shift_t_plus_1 = (0, 0) if time_shift == -1 else (-time_shift, -time_shift)

#     # Merge the predictions while considering the time shift
#     merged_mask = np.logical_or(merged_mask, np.roll(mask_t_minus_1, shift_t_minus_1, axis=(0, 1)))
#     merged_mask = np.logical_or(merged_mask, mask_t)
#     merged_mask = np.logical_or(merged_mask, np.roll(mask_t_plus_1, shift_t_plus_1, axis=(0, 1)))

#     return merged_mask

# Example usage:
# Assume mask_t_minus_1, mask_t, mask_t_plus_1 are numpy arrays representing the masks
# time_shift is an integer representing the time shift (-1, 0, or 1)
# merged_mask = merge_predictions(mask_t_minus_1, mask_t, mask_t_plus_1, time_shift)


def merge_predictions(mask_t_minus_1, mask_t, mask_t_plus_1, time_shift):
    # Ensure all three masks have the same dimensions (1856, 256, 256)
    assert mask_t_minus_1.shape == mask_t.shape == mask_t_plus_1.shape == (1856, 256, 256), "All masks must have shape (1856, 256, 256)"

    # Create an empty merged mask with zeros
    merged_mask = np.zeros((1856, 256, 256))

    # Time shift should be within the bounds of the mask dimensions
    time_shift = max(-1, min(1, time_shift))

    # Calculate the shift directions for t-1, t, and t+1 masks
    shift_t_minus_1 = (0, 0) if time_shift == 1 else (time_shift, time_shift)
    shift_t_plus_1 = (0, 0) if time_shift == -1 else (-time_shift, -time_shift)

    # Merge the predictions while considering the time shift for each frame
    for i in range(1856):
        merged_mask[i] = np.logical_or(merged_mask[i], np.roll(mask_t_minus_1[i], shift_t_minus_1, axis=(0, 1)))
        merged_mask[i] = np.logical_or(merged_mask[i], mask_t[i])
        merged_mask[i] = np.logical_or(merged_mask[i], np.roll(mask_t_plus_1[i], shift_t_plus_1, axis=(0, 1)))

    return merged_mask




In [91]:
for time_shift in range(256):
    x = torch.from_numpy(merge_predictions(x1.cpu().numpy(),x2.cpu().numpy(),x3.cpu().numpy(), time_shift)).cuda()

    score = dice_coef(model_masks[idx], x, thr=0.5).cpu().detach().numpy() 
    print(score)
    best_threshold = 0.0
    best_dice_score = 0.0
    for threshold in [i / 100 for i in range(101)] :
        score = dice_coef(model_masks, x, thr=threshold).cpu().detach().numpy() 
        if score > best_dice_score:
            best_dice_score = score
            best_threshold = threshold
    print(best_dice_score, best_threshold)
        

1.7464229e-12
0.39960405 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.4031229 0.0
1.7643748e-12
0.403

In [83]:
model_masks[0]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')

In [85]:
x1.shape

torch.Size([1856, 256, 256])